In [36]:
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt

def find_project_root(markers=("pyproject.toml", "requirements.txt", ".git", "README.md")) -> Path:
    here = Path.cwd().resolve()
    for candidate in [here, *here.parents]:
        if any((candidate / m).exists() for m in markers):
            return candidate
    return here

ROOT = find_project_root()
data_path = ROOT / "docs" / "07-base-vendas-HP-EliteBook.xlsx"
results_dir = ROOT / "results"
results_dir.mkdir(parents=True, exist_ok=True)

def _parse_number(x):
    if pd.isna(x):
        return np.nan
    if isinstance(x, (int, float, np.integer, np.floating)):
        return float(x)
    s = str(x).strip()
    s = s.replace(".", "").replace(",", ".")
    try:
        return float(s)
    except Exception:
        return np.nan

df = pd.read_excel(data_path)

df.columns = [c.strip() for c in df.columns]

if "Unidades_Vendidas" in df.columns:
    df["Unidades_Vendidas"] = pd.to_numeric(df["Unidades_Vendidas"], errors="coerce").astype("Int64")
if "Preço_Unitário" in df.columns:
    df["Preço_Unitário"] = df["Preço_Unitário"].apply(_parse_number)
if "Receita" in df.columns:
    df["Receita"] = df["Receita"].apply(_parse_number)
if "Ano" in df.columns:
    df["Ano"] = pd.to_numeric(df["Ano"], errors="coerce").astype("Int64")
if "Mês" in df.columns:
    df["Mês"] = pd.to_numeric(df["Mês"], errors="coerce").astype("Int64")

if all(col in df.columns for col in ["Receita", "Unidades_Vendidas", "Preço_Unitário"]):
    mask_null = df["Receita"].isna()
    df.loc[mask_null, "Receita"] = (
        df.loc[mask_null, "Unidades_Vendidas"].astype("float") * df.loc[mask_null, "Preço_Unitário"].astype("float")
    )

if "Ano" in df.columns and "Mês" in df.columns:
    df["AnoMes"] = pd.to_datetime(
        {
            "year": pd.to_numeric(df["Ano"], errors="coerce"),
            "month": pd.to_numeric(df["Mês"], errors="coerce"),
            "day": 1,
        },
        errors="coerce",
    )

preview_path = results_dir / "01_dataset_preview.csv"
df.head(240).to_csv(preview_path, index=False, encoding="utf-8")

print(f"[OK] Raiz do projeto: {ROOT}")
print(f"[OK] Lido de: {data_path}")
print(f"[OK] Resultados em: {results_dir}")
print(f"[OK] Prévia salva em: {preview_path}")
print(f"[INFO] Linhas: {len(df):,} | Colunas: {len(df.columns)}")
df.head()

[OK] Raiz do projeto: C:\Users\Vinícius Andrade\Desktop\data-analysis-with-python
[OK] Lido de: C:\Users\Vinícius Andrade\Desktop\data-analysis-with-python\docs\07-base-vendas-HP-EliteBook.xlsx
[OK] Resultados em: C:\Users\Vinícius Andrade\Desktop\data-analysis-with-python\results
[OK] Prévia salva em: C:\Users\Vinícius Andrade\Desktop\data-analysis-with-python\results\01_dataset_preview.csv
[INFO] Linhas: 240 | Colunas: 16


,Ano,Mês,Produto,Canal,Regiões,Unidades_Vendidas,Preço_Unitário,Receita,Sazonalidade,Faixa Etária 25 a 40 anos,Faixa Etária 30 a 45 anos,Faixa Etária 30+ anos,Faixa Etária 35+ anos,Loja,Funcionario,AnoMes
0,2020,1,"EliteBook 840 G6 (Intel i5, 8GB, 256GB SSD)",Site,Norte,562,5200.0,2922400.0,NaN,169,197,112,84,Loja B,Bruna,2020-01-01
1,2020,1,"EliteBook 840 G8 (Intel i5, 16GB, 512GB SSD)",Site,Nordeste,339,6750.0,2288250.0,NaN,102,119,68,51,Loja B,Lucas,2020-01-01
2,2020,1,"EliteBook 640 G9 (Intel i7, 16GB, 512GB SSD)",Marketplace,Centro-Oeste,555,2300.0,1276500.0,NaN,167,194,111,83,Loja B,Lucas,2020-01-01
3,2020,1,"EliteBook x360 1040 G11 (Intel i7, 32GB, 1TB SSD)",Marketplace,Sudeste,528,9500.0,5016000.0,NaN,158,185,106,79,Loja C,Jéssica,2020-01-01
4,2020,2,"EliteBook 840 G6 (Intel i5, 8GB, 256GB SSD)",Marketplace,Sul,457,5200.0,2376400.0,NaN,137,160,91,69,Loja C,Isabela,2020-02-01


In [37]:
from pathlib import Path
import pandas as pd

if "results_dir" not in locals():
    def find_project_root(markers=("pyproject.toml", "requirements.txt", ".git", "README.md")) -> Path:
        here = Path.cwd().resolve()
        for candidate in [here, *here.parents]:
            if any((candidate / m).exists() for m in markers):
                return candidate
        return here
    ROOT = find_project_root()
    results_dir = ROOT / "results"
    results_dir.mkdir(parents=True, exist_ok=True)

shape_info = pd.DataFrame({"metric": ["rows", "cols"], "value": [df.shape[0], df.shape[1]]})

missing = df.isna().sum().reset_index()
missing.columns = ["coluna", "missing"]

duplicates_count = int(df.duplicated().sum())

unique_counts = df.nunique(dropna=True).reset_index()
unique_counts.columns = ["coluna", "unique_values"]

shape_info.to_csv(results_dir / "02_shape_info.csv", index=False, encoding="utf-8")
missing.to_csv(results_dir / "02_missing_values.csv", index=False, encoding="utf-8")
unique_counts.to_csv(results_dir / "02_unique_counts.csv", index=False, encoding="utf-8")

with (results_dir / "02_data_profile.txt").open("w", encoding="utf-8") as f:
    f.write(f"rows={df.shape[0]} cols={df.shape[1]}\n")
    f.write(f"duplicates={duplicates_count}\n")

print(f"[OK] Perfil de qualidade salvo em: {results_dir}")
shape_info

[OK] Perfil de qualidade salvo em: C:\Users\Vinícius Andrade\Desktop\data-analysis-with-python\results


,metric,value
0,rows,240
1,cols,16


In [38]:
from pathlib import Path
import pandas as pd
import numpy as np

if "results_dir" not in locals():
    def find_project_root(markers=("pyproject.toml", "requirements.txt", ".git", "README.md")) -> Path:
        here = Path.cwd().resolve()
        for candidate in [here, *here.parents]:
            if any((candidate / m).exists() for m in markers):
                return candidate
        return here
    ROOT = find_project_root()
    results_dir = ROOT / "results"
    results_dir.mkdir(parents=True, exist_ok=True)

def _safe_sum(series):
    return float(pd.to_numeric(series, errors="coerce").sum()) if series is not None else np.nan

total_units = _safe_sum(df["Unidades_Vendidas"]) if "Unidades_Vendidas" in df.columns else np.nan
total_rev   = _safe_sum(df["Receita"])           if "Receita" in df.columns           else np.nan

blended_price = (total_rev / total_units) if (
    isinstance(total_units, (int, float, np.integer, np.floating)) and total_units and not np.isnan(total_units)
) else np.nan

kpi = {
    "total_receita": [total_rev],
    "total_unidades": [total_units],
    "preco_medio_ponderado": [blended_price],
    "num_produtos": [df["Produto"].nunique(dropna=True) if "Produto" in df.columns else np.nan],
    "num_canais": [df["Canal"].nunique(dropna=True) if "Canal" in df.columns else np.nan],
    "num_regioes": [df["Regiões"].nunique(dropna=True) if "Regiões" in df.columns else np.nan],
    "num_lojas": [df["Loja"].nunique(dropna=True) if "Loja" in df.columns else np.nan],
    "num_funcionarios": [df["Funcionario"].nunique(dropna=True) if "Funcionario" in df.columns else np.nan],
}
kpi_df = pd.DataFrame(kpi)

kpi_path = results_dir / "03_kpis_gerais.csv"
kpi_df.to_csv(kpi_path, index=False, encoding="utf-8")

print(f"[OK] KPIs salvos em: {kpi_path}")
kpi_df

[OK] KPIs salvos em: C:\Users\Vinícius Andrade\Desktop\data-analysis-with-python\results\03_kpis_gerais.csv


,total_receita,total_unidades,preco_medio_ponderado,num_produtos,num_canais,num_regioes,num_lojas,num_funcionarios
0,808832100.0,133365.0,6064.80036,4,2,5,3,9


In [39]:
from pathlib import Path
import matplotlib.pyplot as plt

if "results_dir" not in locals():
    def find_project_root(markers=("pyproject.toml", "requirements.txt", ".git", "README.md")) -> Path:
        here = Path.cwd().resolve()
        for candidate in [here, *here.parents]:
            if any((candidate / m).exists() for m in markers):
                return candidate
        return here
    ROOT = find_project_root()
    results_dir = ROOT / "results"
    results_dir.mkdir(parents=True, exist_ok=True)

if "AnoMes" in df.columns:
    ts = (
        df.groupby("AnoMes", dropna=True)
          .agg(receita=("Receita", "sum"),
               unidades=("Unidades_Vendidas", "sum"))
          .reset_index()
          .sort_values("AnoMes")
    )

    ts_path = results_dir / "04_timeseries.csv"
    ts.to_csv(ts_path, index=False, encoding="utf-8")
    print(f"[OK] Timeseries salva em: {ts_path}")

    plt.figure()
    plt.plot(ts["AnoMes"], ts["receita"])
    plt.title("Receita por mês")
    plt.xlabel("Ano-Mês")
    plt.ylabel("Receita")
    plt.xticks(rotation=45)
    plt.tight_layout()
    fig1_path = results_dir / "04_timeseries_receita.png"
    plt.savefig(fig1_path, dpi=120)
    plt.close()
    print(f"[OK] Figura salva em: {fig1_path}")

    plt.figure()
    plt.plot(ts["AnoMes"], ts["unidades"])
    plt.title("Unidades vendidas por mês")
    plt.xlabel("Ano-Mês")
    plt.ylabel("Unidades")
    plt.xticks(rotation=45)
    plt.tight_layout()
    fig2_path = results_dir / "04_timeseries_unidades.png"
    plt.savefig(fig2_path, dpi=120)
    plt.close()
    print(f"[OK] Figura salva em: {fig2_path}")
else:
    print("[WARN] Coluna 'AnoMes' ausente. Pule esta análise se não houver (Ano, Mês).")

[OK] Timeseries salva em: C:\Users\Vinícius Andrade\Desktop\data-analysis-with-python\results\04_timeseries.csv
[OK] Figura salva em: C:\Users\Vinícius Andrade\Desktop\data-analysis-with-python\results\04_timeseries_receita.png
[OK] Figura salva em: C:\Users\Vinícius Andrade\Desktop\data-analysis-with-python\results\04_timeseries_unidades.png


In [40]:
from pathlib import Path
import matplotlib.pyplot as plt

if "results_dir" not in locals():
    def find_project_root(markers=("pyproject.toml", "requirements.txt", ".git", "README.md")) -> Path:
        here = Path.cwd().resolve()
        for candidate in [here, *here.parents]:
            if any((candidate / m).exists() for m in markers):
                return candidate
        return here
    ROOT = find_project_root()
    results_dir = ROOT / "results"
    results_dir.mkdir(parents=True, exist_ok=True)

if "Produto" in df.columns:
    prod = (
        df.groupby("Produto", dropna=True)
          .agg(
              receita=("Receita", "sum"),
              unidades=("Unidades_Vendidas", "sum"),
              preco_medio=("Preço_Unitário", "mean"),
          )
          .reset_index()
          .sort_values("receita", ascending=False)
    )

    total_receita = prod["receita"].sum()
    prod["share_receita_%"] = (prod["receita"] / total_receita * 100.0) if total_receita else 0.0

    prod_path = results_dir / "05_produto_desempenho.csv"
    prod.to_csv(prod_path, index=False, encoding="utf-8")
    print(f"[OK] Tabela produto salva em: {prod_path}")

    top = prod.head(10)
    plt.figure()
    plt.bar(top["Produto"], top["receita"])
    plt.title("Receita por Produto (Top 10)")
    plt.ylabel("Receita")
    plt.xticks(rotation=45, ha="right")
    plt.tight_layout()
    fig_path = results_dir / "05_produto_receita_top10.png"
    plt.savefig(fig_path, dpi=120)
    plt.close()
    print(f"[OK] Figura salva em: {fig_path}")
else:
    print("[WARN] Coluna 'Produto' ausente.")

[OK] Tabela produto salva em: C:\Users\Vinícius Andrade\Desktop\data-analysis-with-python\results\05_produto_desempenho.csv
[OK] Figura salva em: C:\Users\Vinícius Andrade\Desktop\data-analysis-with-python\results\05_produto_receita_top10.png


In [41]:
from pathlib import Path
import matplotlib.pyplot as plt

if "results_dir" not in locals():
    def find_project_root(markers=("pyproject.toml", "requirements.txt", ".git", "README.md")) -> Path:
        here = Path.cwd().resolve()
        for candidate in [here, *here.parents]:
            if any((candidate / m).exists() for m in markers):
                return candidate
        return here
    ROOT = find_project_root()
    results_dir = ROOT / "results"
    results_dir.mkdir(parents=True, exist_ok=True)

if "Canal" in df.columns:
    canal = (
        df.groupby("Canal", dropna=True)
          .agg(
              receita=("Receita", "sum"),
              unidades=("Unidades_Vendidas", "sum")
          )
          .reset_index()
          .sort_values("receita", ascending=False)
    )

    # Participação de receita por canal
    total_receita = canal["receita"].sum()
    canal["share_receita_%"] = (canal["receita"] / total_receita * 100.0) if total_receita else 0.0

    # Persistência
    canal_path = results_dir / "06_canal_mix.csv"
    canal.to_csv(canal_path, index=False, encoding="utf-8")
    print(f"[OK] Canal mix salvo em: {canal_path}")

    # Gráfico: Receita por Canal
    plt.figure()
    plt.bar(canal["Canal"], canal["receita"])
    plt.title("Receita por Canal")
    plt.ylabel("Receita")
    plt.xticks(rotation=45, ha="right")
    plt.tight_layout()
    fig_path = results_dir / "06_canal_receita.png"
    plt.savefig(fig_path, dpi=120)
    plt.close()
    print(f"[OK] Figura salva em: {fig_path}")
else:
    print("[WARN] Coluna 'Canal' ausente.")

[OK] Canal mix salvo em: C:\Users\Vinícius Andrade\Desktop\data-analysis-with-python\results\06_canal_mix.csv
[OK] Figura salva em: C:\Users\Vinícius Andrade\Desktop\data-analysis-with-python\results\06_canal_receita.png


In [42]:
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt

if "results_dir" not in locals():
    def find_project_root(markers=("pyproject.toml", "requirements.txt", ".git", "README.md")) -> Path:
        here = Path.cwd().resolve()
        for candidate in [here, *here.parents]:
            if any((candidate / m).exists() for m in markers):
                return candidate
        return here
    ROOT = find_project_root()
    results_dir = ROOT / "results"
    results_dir.mkdir(parents=True, exist_ok=True)

if "Regiões" in df.columns:
    reg = (
        df.groupby("Regiões", dropna=True)
          .agg(
              receita=("Receita", "sum"),
              unidades=("Unidades_Vendidas", "sum"),
          )
          .reset_index()
          .sort_values("receita", ascending=False)
    )

    reg_path = results_dir / "07_regiao_distribuicao.csv"
    reg.to_csv(reg_path, index=False, encoding="utf-8")
    print(f"[OK] Distribuição regional salva em: {reg_path}")

    plt.figure()
    plt.bar(reg["Regiões"], reg["receita"])
    plt.title("Receita por Região")
    plt.ylabel("Receita")
    plt.xticks(rotation=45, ha="right")
    plt.tight_layout()
    fig_path = results_dir / "07_regiao_receita.png"
    plt.savefig(fig_path, dpi=120)
    plt.close()
    print(f"[OK] Figura salva em: {fig_path}")
else:
    print("[WARN] Coluna 'Regiões' ausente.")

[OK] Distribuição regional salva em: C:\Users\Vinícius Andrade\Desktop\data-analysis-with-python\results\07_regiao_distribuicao.csv
[OK] Figura salva em: C:\Users\Vinícius Andrade\Desktop\data-analysis-with-python\results\07_regiao_receita.png


In [43]:
# 8) Relação Preço x Unidades (correlação e dispersão) — paths relativos
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Usa o results_dir da Célula 1; se executar isolado, cria a partir da raiz do projeto
if "results_dir" not in locals():
    def find_project_root(markers=("pyproject.toml", "requirements.txt", ".git", "README.md")) -> Path:
        here = Path.cwd().resolve()
        for candidate in [here, *here.parents]:
            if any((candidate / m).exists() for m in markers):
                return candidate
        return here
    ROOT = find_project_root()
    results_dir = ROOT / "results"
    results_dir.mkdir(parents=True, exist_ok=True)

if "Preço_Unitário" in df.columns and "Unidades_Vendidas" in df.columns:
    aux = (
        df[["Preço_Unitário", "Unidades_Vendidas"]]
        .copy()
        .apply(pd.to_numeric, errors="coerce")
        .dropna()
    )

    corr = aux.corr(method="pearson")
    corr_path = results_dir / "08_correlacao_preco_unidades.csv"
    corr.to_csv(corr_path, encoding="utf-8")
    print(f"[OK] Correlação salva em: {corr_path}")

    plt.figure()
    plt.scatter(aux["Preço_Unitário"], aux["Unidades_Vendidas"])
    plt.title("Dispersão: Preço Unitário x Unidades Vendidas")
    plt.xlabel("Preço Unitário")
    plt.ylabel("Unidades Vendidas")
    plt.tight_layout()
    fig_path = results_dir / "08_scatter_preco_vs_unidades.png"
    plt.savefig(fig_path, dpi=120)
    plt.close()
    print(f"[OK] Figura salva em: {fig_path}")

    print("[INFO] Pearson r:",
          float(corr.loc["Preço_Unitário", "Unidades_Vendidas"]) if
          "Preço_Unitário" in corr.index and "Unidades_Vendidas" in corr.columns else np.nan)
else:
    print("[WARN] Colunas 'Preço_Unitário' e/ou 'Unidades_Vendidas' ausentes.")

[OK] Correlação salva em: C:\Users\Vinícius Andrade\Desktop\data-analysis-with-python\results\08_correlacao_preco_unidades.csv
[OK] Figura salva em: C:\Users\Vinícius Andrade\Desktop\data-analysis-with-python\results\08_scatter_preco_vs_unidades.png
[INFO] Pearson r: 0.1032520503107852


In [44]:
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

if "results_dir" not in locals():
    def find_project_root(markers=("pyproject.toml", "requirements.txt", ".git", "README.md")) -> Path:
        here = Path.cwd().resolve()
        for candidate in [here, *here.parents]:
            if any((candidate / m).exists() for m in markers):
                return candidate
        return here
    ROOT = find_project_root()
    results_dir = ROOT / "results"
    results_dir.mkdir(parents=True, exist_ok=True)

faixas = [
    "Faixa Etária 25 a 40 anos",
    "Faixa Etária 30 a 45 anos",
    "Faixa Etária 30+ anos",
    "Faixa Etária 35+ anos",
]
present = [c for c in faixas if c in df.columns]

if present:
    agg = (
        df[present]
        .apply(pd.to_numeric, errors="coerce")
        .sum()
        .reset_index()
    )
    agg.columns = ["faixa", "unidades"]

    total = float(agg["unidades"].sum())
    agg["participacao_%"] = (agg["unidades"] / total * 100.0) if total else np.nan

    out_path = results_dir / "09_faixas_etarias.csv"
    agg.to_csv(out_path, index=False, encoding="utf-8")
    print(f"[OK] Tabela de faixas etárias salva em: {out_path}")

    plt.figure()
    plt.bar(agg["faixa"], agg["unidades"])
    plt.title("Unidades por Faixa Etária")
    plt.ylabel("Unidades")
    plt.xticks(rotation=45, ha="right")
    plt.tight_layout()
    fig_path = results_dir / "09_faixas_etarias_barras.png"
    plt.savefig(fig_path, dpi=120)
    plt.close()
    print(f"[OK] Figura salva em: {fig_path}")

    agg
else:
    print("[WARN] Colunas de faixas etárias não encontradas:", faixas)

[OK] Tabela de faixas etárias salva em: C:\Users\Vinícius Andrade\Desktop\data-analysis-with-python\results\09_faixas_etarias.csv
[OK] Figura salva em: C:\Users\Vinícius Andrade\Desktop\data-analysis-with-python\results\09_faixas_etarias_barras.png


In [45]:
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt

if "results_dir" not in locals():
    def find_project_root(markers=("pyproject.toml", "requirements.txt", ".git", "README.md")) -> Path:
        here = Path.cwd().resolve()
        for candidate in [here, *here.parents]:
            if any((candidate / m).exists() for m in markers):
                return candidate
        return here
    ROOT = find_project_root()
    results_dir = ROOT / "results"
    results_dir.mkdir(parents=True, exist_ok=True)

if "Loja" in df.columns:
    loja = (
        df.groupby("Loja", dropna=True)
          .agg(
              receita=("Receita", "sum"),
              unidades=("Unidades_Vendidas", "sum")
          )
          .reset_index()
          .sort_values("receita", ascending=False)
    )
    loja_path = results_dir / "10_loja_desempenho.csv"
    loja.to_csv(loja_path, index=False, encoding="utf-8")
    print(f"[OK] Loja desempenho salvo em: {loja_path}")

    top_l = loja.head(10)
    plt.figure()
    plt.bar(top_l["Loja"], top_l["receita"])
    plt.title("Receita por Loja (Top 10)")
    plt.ylabel("Receita")
    plt.xticks(rotation=45, ha="right")
    plt.tight_layout()
    fig_loja = results_dir / "10_loja_receita_top10.png"
    plt.savefig(fig_loja, dpi=120)
    plt.close()
    print(f"[OK] Figura salva em: {fig_loja}")
else:
    print("[WARN] Coluna 'Loja' ausente.")

if "Funcionario" in df.columns:
    func = (
        df.groupby("Funcionario", dropna=True)
          .agg(
              receita=("Receita", "sum"),
              unidades=("Unidades_Vendidas", "sum")
          )
          .reset_index()
          .sort_values("receita", ascending=False)
    )
    func_path = results_dir / "10_funcionario_desempenho.csv"
    func.to_csv(func_path, index=False, encoding="utf-8")
    print(f"[OK] Funcionário desempenho salvo em: {func_path}")

    top_f = func.head(10)
    plt.figure()
    plt.bar(top_f["Funcionario"], top_f["receita"])
    plt.title("Receita por Funcionário (Top 10)")
    plt.ylabel("Receita")
    plt.xticks(rotation=45, ha="right")
    plt.tight_layout()
    fig_func = results_dir / "10_funcionario_receita_top10.png"
    plt.savefig(fig_func, dpi=120)
    plt.close()
    print(f"[OK] Figura salva em: {fig_func}")
else:
    print("[WARN] Coluna 'Funcionario' ausente.")

[OK] Loja desempenho salvo em: C:\Users\Vinícius Andrade\Desktop\data-analysis-with-python\results\10_loja_desempenho.csv
[OK] Figura salva em: C:\Users\Vinícius Andrade\Desktop\data-analysis-with-python\results\10_loja_receita_top10.png
[OK] Funcionário desempenho salvo em: C:\Users\Vinícius Andrade\Desktop\data-analysis-with-python\results\10_funcionario_desempenho.csv
[OK] Figura salva em: C:\Users\Vinícius Andrade\Desktop\data-analysis-with-python\results\10_funcionario_receita_top10.png
